# CityBikes

### Set-up

In [244]:
# import libraries
import requests
import pandas as pd
import time

In [245]:
# set constants
ROOT_URL = 'http://api.citybik.es/v2/networks'

In [293]:
# define functions

# 'networks' functions
def getNetworks_url(filter_fields = []):
    if len(filter_fields) > 0:
        filter_fields_string = ''
        for field in filter_fields:
            filter_fields_string += '{},'.format(field)
        return ROOT_URL + '?fields={}'.format(filter_fields_string)
    return ROOT_URL

def getNetworks_df(filter_fields = []):
    response = requests.get(getNetworks_url(filter_fields)).json()
    networks_df = pd.json_normalize(response['networks'])
    return networks_df

# 'network' functions
def getNetwork_id(city = ''):
    response = requests.get(getURL_filter(['location', 'id'])).json()
    networks_df = pd.json_normalize(response['networks'])
    id_df = networks_df[networks_df['location.city'] == city]
    network_id = id_df.iloc[0]['id']
    return network_id #Needs error handling for if city is a empty string

def getNetwork_df(city = ''):
    response = requests.get(ROOT_URL + '/' + getNetwork_id(city) + '?fields=stations').json()
    network_df = pd.json_normalize(response['network']['stations'], max_level = 0)
    return network_df

# project specific functions
def getCityBikes_df(df = pd.DataFrame()):
    cityBikes_df = pd.DataFrame()
    cityBikes_df['bikes'] = df['empty_slots'] + df['free_bikes']
    cityBikes_df['latitude'] = df['latitude']
    cityBikes_df['longitude'] = df['longitude']
    return cityBikes_df # needs error handling for if df is empty DataFrame

### Exploring Networks

In [239]:
# parameters
filter_fields = [] # options are [name, id, href, location]

In [243]:
getNetworks_df().head()

,company,href,id,name,location.city,location.country,location.latitude,location.longitude,source,gbfs_href,license.name,license.url,ebikes
0,[ЗАО «СитиБайк»],/v2/networks/velobike-moscow,velobike-moscow,Velobike,Moscow,RU,55.750000,37.616667,NaN,NaN,NaN,NaN,NaN
1,[Urban Infrastructure Partner],/v2/networks/baerum-bysykkel,baerum-bysykkel,Bysykkel,Bærum,NO,59.894550,10.546343,NaN,NaN,NaN,NaN,NaN
2,[Comunicare S.r.l.],/v2/networks/bicincitta-siena,bicincitta-siena,Bicincittà,Siena,IT,43.318600,11.330600,https://www.bicincitta.com/frmLeStazioni.aspx?...,NaN,NaN,NaN,NaN
3,[Cyclopolis Systems],/v2/networks/cyclopolis-maroussi,cyclopolis-maroussi,Cyclopolis,Maroussi,GR,38.056872,23.808330,NaN,NaN,NaN,NaN,NaN
4,"[Groundwork, Slough Borough Council, ITS]",/v2/networks/cycle-hire-slough,cycle-hire-slough,Cycle Hire,Slough,GB,51.511350,-0.591562,NaN,NaN,NaN,NaN,NaN


#### Functionality to integrate:
For getNetworks_df output:
- networks_df.rename(columns = {'location.city' : 'city', 'location.country' : 'country'}
- networks_df.drop(columns = ['href', 'source', 'gbfs_href', 'license.name', 'license.url'])

Explore the possibility of adding parameter fields like in Google Colab

### Exploring Network

In [287]:
# parameters
city = 'Barcelona'

In [288]:
getNetwork_df(city).head()

,empty_slots,extra,free_bikes,id,latitude,longitude,name,timestamp
0,24,"{'address': 'CARRER DE LA MARINA,25-33', 'alti...",19,8b92416c1f4a49323575c4ce26f0fc8c,41.388125,2.195187,"CARRER DE LA MARINA,25-33",2023-02-25T19:11:50.393000Z
1,19,"{'address': 'C/ ONZE DE SETEMBRE, 37-39', 'alt...",7,ad095f2d769ae6eb8ce76db4bde21a0c,41.429861,2.192201,"C/ ONZE DE SETEMBRE, 37-39",2023-02-25T19:11:50.682000Z
2,15,"{'address': 'PG. FABRA I PUIG, 385', 'altitude...",13,0f850a848dad56671cf211f1684580b8,41.433326,2.162926,"PG. FABRA I PUIG, 385",2023-02-25T19:11:51.176000Z
3,7,"{'address': 'C / PROVENÇA, 241', 'altitude': 3...",11,9a40a01fc442d4f03d0c825280e7f70e,41.393886,2.160164,"C / PROVENÇA, 241",2023-02-25T19:11:50.425000Z
4,20,"{'address': 'C/ BRUC, 130', 'altitude': 34.0, ...",8,152ba80b3ff3a4495a001e4e2ee3e50e,41.396984,2.166185,"C/ BRUC, 130",2023-02-25T19:11:50.589000Z


#### Functionality to integrate:
- Error handling for getNetworks_id()

Explore the possibility of adding parameter fields like in Google Colab

### Acquiring project specific data

In [300]:
# parameters
city = 'Barcelona'

In [301]:
getCityBikes_df(getNetwork_df(city))

,bikes,latitude,longitude
0,43,41.388125,2.195187
1,26,41.429861,2.192201
2,28,41.433326,2.162926
3,18,41.393886,2.160164
4,28,41.396984,2.166185
...,...,...,...
500,18,41.407846,2.158670
501,24,41.392878,2.143411
502,16,41.405270,2.142020
503,34,41.411075,2.180976


#### Functionality to integrate:
- Error handling for getCityBikes()

Explore the possibility of adding parameter fields like in Google Colab